In [2]:
# Importar pandas e numpy para trabalhar com Dataframes
import pandas as pd
import numpy as np

# Importar pyarrow para ler arquivo parquet
import pyarrow as pa
import  pyarrow.parquet as pq

In [3]:
# Definir o caminho em que se encontra o dataset em parquet
path_in = "../../Datasets/MICRODADOS_ENEM_2019_SAOPAULO_FILTERED.parquet"

In [4]:
# Ler o arquivo do dataset
df = pd.read_parquet(path_in)

In [5]:
def vector_to_columns(df, column_prefix, column_name, qtd_columns):
    '''
    Expande a coluna de um dataframe constituida por um vetor
    de strings em uma coluna para cada elemento do vetor. 

    Parameters:
        df (dataframe): dataframe que será expandido
        column_prefix (str): prefixo que será colocado no nome das novas colunas
        column_name (str): nome das novas colunas
        qtd_columns (int): quantidade de novas colunas 

    Returns:
        dataframe: Retorna o dataframe expandido
    '''    

    new_df = df[column_prefix + column_name].str.split("",0,expand=True)
    new_df.drop(columns=[0, qtd_columns], axis="columns", inplace=True)

    return new_df


def remove_extra_columns(dfRespostas, dfGabarito):
    '''
    Remove as colunas adicionais para a prova de LC,
    referentes às questôes da lingua que não foi escolhida
    pelo participante  

    Parameters:
        dfRespostas (dataframe): Dataframe contendo as respostas selecionadas
        dfGabarito (dataframe): Dataframe contendo o gabarito das questões

    Returns:
        dfRespostas, dfGabarito: Retorna os dataframes atualizados
    ''' 

    filter = dfRespostas[1] == "9"

    for i in range(1,6):
        dfRespostas.loc[filter, i] = dfRespostas[i+5]
        dfGabarito.loc[filter, i] = dfGabarito[i+5]

    dfRespostas.drop(columns=[6,7,8,9,10], inplace=True)
    dfGabarito.drop(columns=[6,7,8,9,10], inplace=True)

    return dfRespostas, dfGabarito


def alternative_to_boolean(dfRespostas, dfGabarito, columnsNames):
    '''
    Converte a resposta do candidato (Entre 'A' e 'E') em um valor
    booleano indicando se acertou (1) ou errou (0) a questão

    Parameters:
        dfRespostas (dataframe): Dataframe contendo as respostas selecionadas
        dfGabarito (dataframe): Dataframe contendo o gabarito das questões
        columnsNames (arr): Array contendo os nomes das colunas de respostas

    Returns:
        dfRespostas: Retorna o dataframe de respostas atualizado
    ''' 

    for column in columnsNames:
        conditions = [dfRespostas[column] == dfGabarito[column], pd.isna(dfRespostas[column])]
        choices = [1, np.nan]
        dfRespostas[column] = np.select(conditions, choices, default=0)
    
    return dfRespostas


In [ ]:
# Criar o dicionário contendo os dataframes vazios para cada area
dictSubjects = {"CN":[], "MT":[], "CH":[], "LC":[]}

# Executa as instruções abaixo para cada uma das areas
for subject in dictSubjects.keys():

    # Quantidade de colunas a serem criadas para as respostas
    qtd_columns = 46 if subject != "LC" else 51

    # Filtrar o dataset original e dividir entre dataset de respostas e gabarito
    # convertendo as colunas com vetor de respostas e do gabarito
    dfSubject = df[["NU_INSCRICAO", "TX_RESPOSTAS_" + subject, "TX_GABARITO_" + subject]].dropna()
    dfRespostas = vector_to_columns(dfSubject, "TX_RESPOSTAS_", subject, qtd_columns)
    dfGabarito = vector_to_columns(dfSubject, "TX_GABARITO_", subject, qtd_columns)
    
    # Definir colunas que serão unidas no dataset de respostas
    # Se a area for "LC", é adocionada mais uma coluna e as colunas extras são removidas
    columnsToJoin = ["NU_INSCRICAO", "NU_NOTA_" + subject, "CO_PROVA_" + subject]

    if subject == "LC":
        dfRespostas, dfGabarito = remove_extra_columns(dfRespostas, dfGabarito)
        qtd_columns = 46
        columnsToJoin.append("TP_LINGUA")

    # Definir vetor contendo os nomes das novas colunas,
    # com o padrao "{area}_{numero da questao}"
    columnsNames = [subject + str(i) for i in range(1,qtd_columns)]
    dfRespostas.columns = columnsNames
    dfGabarito.columns = columnsNames

    # Converter as colunas de str para boolean, indicando se acertou ou errou a questao
    # Inserir a coluna "NU_INSCRICAO" e remover as notas 0
    dictSubjects[subject] = alternative_to_boolean(dfRespostas, dfGabarito, columnsNames)
    dictSubjects[subject].insert(0, "NU_INSCRICAO", dfSubject["NU_INSCRICAO"])
    dictSubjects[subject] = dictSubjects[subject].join(df[columnsToJoin].set_index("NU_INSCRICAO"), on="NU_INSCRICAO", how="left")
    dictSubjects[subject] = dictSubjects[subject][dictSubjects[subject]["NU_NOTA_" + subject]!=0]

In [9]:
# Salvar os datasets divididos por area
for subject, dfSubject in dictSubjects.items():
    
    table = pa.Table.from_pandas(dfSubject.reset_index(drop=True))
    
    # Definir onde será salvo o novo dataset filtrado por área
    path = f'../../Datasets/MICRODADOS_ENEM_2019_SAOPAULO_BySubject/{subject}.parquet'

    # Gerar um parquet da tabela
    pq.write_to_dataset(
        table,
        root_path=path,
    )
    